In [1]:
!pip install pyomo
import numpy as np
from scipy.stats import norm, uniform
from pyomo.environ import *

Also past

"conda install -c conda-forge glpk"

in anaconda promt to install glpk solver

**Question 1 & 2**

In [2]:
sample_sizes_LB = 30
sample_sizes_UB = 100
M_size = 5
T_size = 5
confidence_level = 0.95
z_score = norm.ppf(confidence_level + (1 - confidence_level) / 2)

In [3]:
def generate_parameters(size):
    w1 = uniform.rvs(1/2, 5, size)
    w2 = uniform.rvs(1/6, 3, size)
    return w1, w2

def solve_optimization_LB(parameters, size):
    model = ConcreteModel()
    model.x1 = Var(domain=NonNegativeReals)
    model.x2 = Var(domain=NonNegativeReals)
    model.y1 = Var(range(size), domain=NonNegativeReals)
    model.y2 = Var(range(size), domain=NonNegativeReals)
    obj_expr = 2*model.x1 + model.x2 + sum(model.y1[s] + model.y2[s] for s in range(size)) / size
    model.obj = Objective(expr=obj_expr, sense=minimize)
    model.constraints = ConstraintList()

    w1, w2 = parameters
    for s in range(size):
        model.constraints.add(expr=model.y1[s] >= 7 - w1[s] * model.x1 - model.x2)
        model.constraints.add(expr=model.y2[s] >= 4 - w2[s] * model.x1 - model.x2)

    solver = SolverFactory('glpk')
    solver.solve(model)
    
    return model


In [4]:
results = {}
all_objectives = []
lower_bounds = []

for i in range(M_size):  
    v = []
    w1, w2 = generate_parameters(sample_sizes_LB)
    model = solve_optimization_LB((w1, w2), sample_sizes_LB)
    x1_value_lb = model.x1.value
    x2_value_lb = model.x2.value
    objective_value = value(model.obj)
    v.append(objective_value)
    all_objectives.extend(v)
    mean_objective = np.mean(v)
  
    results[i] = {
        'mean_objective': mean_objective,
        'x1': x1_value_lb,
        'x2': x2_value_lb
    }
    lower_bounds.append(mean_objective)
    
    print(f"\nReplication {i + 1} Results:")
    print("x1:", model.x1.value)
    print("x2:", model.x2.value)
    print("y1:")
    for s in range(sample_sizes_LB):
        print(f"  {model.y1[s].value}")
    print("y2:")
    for s in range(sample_sizes_LB):
        print(f"  {model.y2[s].value}")


print("\nObjective Function Values for Each Replication:")
for i, mean_objective in results.items():
    print(f"  Replication {i + 1}: {mean_objective}")


print("\nLowes value for All Replications:")
print(f"  Lowes value: {min(lower_bounds)}")

average_objective = np.mean(all_objectives)
print(f"\nAverage Objective Function Value: {average_objective}")

std_dev = np.std(all_objectives, ddof=1)
margin_of_error = z_score * std_dev / np.sqrt(M_size)  
lower_bound_ci = average_objective - margin_of_error
upper_bound_ci = average_objective + margin_of_error
print(f"Confidence Interval (CI) for Average Objective Function Value: ({lower_bound_ci}, {upper_bound_ci})")



Replication 1 Results:
x1: 1.08319751872129
x2: 2.51020689486024
y1:
  3.4685943734365
  0.0
  0.0
  1.54769700399201
  0.868481604436532
  1.5468630848661
  3.43738948327036
  2.12284091440202
  0.0
  2.28880591292343
  0.66569669301303
  0.0
  0.821213803452018
  2.16421574838547
  3.57752516751311
  2.66476358294438
  0.628491748117448
  3.02532439917671
  0.994745192263997
  1.91297937005639
  1.35061394817289
  0.0
  0.0
  0.0
  3.91330681105838
  2.16799095787729
  0.0
  2.37413417054228
  0.00326681805184306
  1.74155444131239
y2:
  0.762966700123112
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0684680774394928
  0.0
  0.0
  0.134512172877615
  0.0
  0.918917828328659
  0.0
  0.0
  0.0
  0.0
  0.0
  0.195461686484812
  0.0
  0.279498850200491
  0.0
  1.2993451866418
  0.0
  0.0
  0.0
  0.0

Replication 2 Results:
x1: 0.819740905528957
x2: 3.39398702917328
y1:
  0.0
  0.0
  0.0
  0.0
  2.99512692982051
  2.71865077844212
  1.62071362203981
  0.759236356701729
  1.14

 **Answer for Question 1 (in one of the code runs) for lower bound** 




Average Objective Function Value: 6.224194052151148



Objective Function Values for Each Replication:


 Replication 1: {'mean_objective': 6.362667396229095, 'x1': 0.89852071605596, 'x2': 3.20736419588154}
 
  Replication 2: {'mean_objective': 6.085607974008829, 'x1': 0.936387513009016, 'x2': 3.08640216784603}
  
  Replication 3: {'mean_objective': 6.249890785547092, 'x1': 1.08781832773417, 'x2': 2.49552782757638}
  
  Replication 4: {'mean_objective': 6.366551890709118, 'x1': 0.770764837347872, 'x2': 3.4893394073101}
  
  Replication 5: {'mean_objective': 6.056252214261607, 'x1': 0.893575796727991, 'x2': 3.06873040402537}
  





**Answer for Question 2 (in one of code the runs) for lower bound**


Confidence Interval (CI) for Average Objective Function Value: (6.094546097302824, 6.353842006999472)

In [5]:
def solve_optimization_UB(parameters, size, x1_value, x2_value):
    model = ConcreteModel()
    model.x1 = Var(domain=NonNegativeReals, initialize=x1_value)
    model.x2 = Var(domain=NonNegativeReals, initialize=x2_value)
    model.y1 = Var(range(size), domain=NonNegativeReals)
    model.y2 = Var(range(size), domain=NonNegativeReals)
    obj_expr = 2 * model.x1 + model.x2 + sum(model.y1[s] + model.y2[s] for s in range(size)) / size
    model.obj = Objective(expr=obj_expr, sense=minimize)
    model.constraints = ConstraintList()

    w1, w2 = parameters
    for s in range(size):
        model.constraints.add(expr=model.y1[s] >= 7 - w1[s] * model.x1 - model.x2)
        model.constraints.add(expr=model.y2[s] >= 4 - w2[s] * model.x1 - model.x2)

    solver = SolverFactory('glpk')
    solver.solve(model)
    
    return model

In [6]:
results_ub = {}
all_objectives_ub = []  
detailed_ub_objectives = [] 
ub_cis = []  
upper_bounds = []  

for i in range(M_size):
    x1_value_lb = results[i]['x1']
    x2_value_lb = results[i]['x2']
    objectives_for_current_lb = []  

    for j in range(T_size):
        w1, w2 = generate_parameters(sample_sizes_UB)
        ub_model = solve_optimization_UB((w1, w2), sample_sizes_UB, x1_value_lb, x2_value_lb)
        objective_value_ub = value(ub_model.obj)
        objectives_for_current_lb.append(objective_value_ub)  
        detailed_ub_objectives.append((i, j, objective_value_ub))  

        print(f"\nLB Replication {i + 1}, UB Replication {j + 1} Results (Objective: {objective_value_ub}):")
        print(f"x1 (from LB): {x1_value_lb}, x2 (from LB): {x2_value_lb}")
        for s in range(sample_sizes_UB):
            print(f"  y1 UB {s + 1}: {ub_model.y1[s].value}, y2 UB {s + 1}: {ub_model.y2[s].value}")

    all_objectives_ub.extend(objectives_for_current_lb) 
    
    std_dev_ub = np.std(objectives_for_current_lb, ddof=1)
    margin_of_error_ub = z_score * std_dev_ub / np.sqrt(len(objectives_for_current_lb))
    lower_bound_ci_ub = np.mean(objectives_for_current_lb) - margin_of_error_ub
    upper_bound_ci_ub = np.mean(objectives_for_current_lb) + margin_of_error_ub
    ub_cis.append((i, lower_bound_ci_ub, upper_bound_ci_ub))

    mean_objective_ub = np.mean(objectives_for_current_lb)
    results_ub[i] = mean_objective_ub
    upper_bounds.append(mean_objective_ub)

# LB Optimization Results
print("\n--- LB Optimization Results ---")
print(f"Lowest Objective Value for All LB Replications: {min(lower_bounds)}")
average_objective_lb = np.mean(lower_bounds)
std_dev_lb = np.std(lower_bounds, ddof=1)
margin_of_error_lb = z_score * std_dev_lb / np.sqrt(M_size)
print(f"Average Objective Function Value (LB Problem): {average_objective_lb}")
print(f"Confidence Interval (CI) for Average Objective Function Value (LB Problem): ({average_objective_lb - margin_of_error_lb}, {average_objective_lb + margin_of_error_lb})")

print("\n--- UB Optimization Results ---")
average_ub_per_lb = {}  


for lb_rep, ub_rep, objective in detailed_ub_objectives:
    print(f"Objective Value for LB Replication {lb_rep + 1}, UB Replication {ub_rep + 1}: {objective}")
    
   
    if lb_rep not in average_ub_per_lb:
        average_ub_per_lb[lb_rep] = []
    average_ub_per_lb[lb_rep].append(objective)

print("\nAverage Objective Values for UB Replications in Each LB Batch:")
for lb_rep, objectives in average_ub_per_lb.items():
    average_obj = np.mean(objectives)
    print(f"Average Objective  of UB, in LB Replication {lb_rep + 1}: {average_obj}")

print("\nConfidence Intervals for UB Optimizations:")
for lb_rep, lower_ci, upper_ci in ub_cis:
    print(f"CI for average of UB, in LB Replication {lb_rep + 1}: ({lower_ci}, {upper_ci})")


LB Replication 1, UB Replication 1 Results (Objective: 6.124771977887719):
x1 (from LB): 1.08319751872129, x2 (from LB): 2.51020689486024
  y1 UB 1: 1.84288548806516, y2 UB 1: 0.643732496836316
  y1 UB 2: 1.99393207629602, y2 UB 2: 0.0
  y1 UB 3: 0.161297698357057, y2 UB 3: 0.0
  y1 UB 4: 0.16499750555583, y2 UB 4: 0.0
  y1 UB 5: 1.07382548191292, y2 UB 5: 0.713932785792943
  y1 UB 6: 0.326725506793846, y2 UB 6: 0.0
  y1 UB 7: 0.174126066348005, y2 UB 7: 0.0
  y1 UB 8: 0.980307344434751, y2 UB 8: 0.0
  y1 UB 9: 0.0, y2 UB 9: 0.129227203392888
  y1 UB 10: 2.63791272950176, y2 UB 10: 0.0
  y1 UB 11: 2.97382261535257, y2 UB 11: 0.685059677832769
  y1 UB 12: 0.859082224945777, y2 UB 12: 0.0
  y1 UB 13: 0.0, y2 UB 13: 0.0
  y1 UB 14: 0.491577501512196, y2 UB 14: 0.0
  y1 UB 15: 0.0, y2 UB 15: 0.731024400848203
  y1 UB 16: 0.0, y2 UB 16: 0.413176485588529
  y1 UB 17: 2.62134135906398, y2 UB 17: 0.505590105129
  y1 UB 18: 2.92632515295759, y2 UB 18: 0.0
  y1 UB 19: 2.24486297548092, y2 UB 19

**Answer for Question 1 (in one of the runs) for Upper bound** 

Average Objective Values for UB Replications in Each LB Batch:

Average Objective  of UB, in LB Replication 1: 6.367248469294919

Average Objective  of UB, in LB Replication 2: 6.336854661248106

Average Objective  of UB, in LB Replication 3: 6.279240496640921

Average Objective  of UB, in LB Replication 4: 6.371429915447083

Average Objective  of UB, in LB Replication 5: 6.276842193163727


**Answer for Question 2 (in one of the runs) for Upper bound**

Confidence Intervals for UB Optimizations:

CI for average of UB, in LB Replication 1: (6.258999145654333, 6.4754977929355055)

CI for average of UB, in LB Replication 2: (6.238094863215723, 6.4356144592804885)

CI for average of UB, in LB Replication 3: (6.16681871071998, 6.391662282561862)

CI for average of UB, in LB Replication 4: (6.2659536462473095, 6.476906184646856)

CI for average of UB, in LB Replication 5: (6.098429273965486, 6.455255112361968)


**Question 3**

In [7]:
def ub_lb_gaps(average_ub_per_lb, lower_bounds):
    
    gaps = []

    print("\nGaps between Average UB Objectives and Average LB Objective:")

    for lb_rep, ub_objectives in average_ub_per_lb.items():
        average_ub = np.mean(ub_objectives)
        gap = average_ub - average_objective_lb
        gaps.append(gap)

        print(f"Gap between UB and LB in Replication {lb_rep + 1}: {gap}")

    return gaps

gaps = ub_lb_gaps(average_ub_per_lb, lower_bounds)


Gaps between Average UB Objectives and Average LB Objective:
Gap between UB and LB in Replication 1: -0.11036719024469743
Gap between UB and LB in Replication 2: 0.018548383024056037
Gap between UB and LB in Replication 3: -0.10407192895500117
Gap between UB and LB in Replication 4: -0.012820034242325917
Gap between UB and LB in Replication 5: -0.031747196100978314


**Question 3 answer (in one of the code runs)**

Gaps between Average UB Objectives and Average LB Objective:

Gap between UB and LB in Replication 1: 0.14305441714377132

Gap between UB and LB in Replication 2: 0.11266060909695774

Gap between UB and LB in Replication 3: 0.05504644448977292

Gap between UB and LB in Replication 4: 0.14723586329593452

Gap between UB and LB in Replication 5: 0.052648141012578975

**Question 4 (changing the sample size)**

In [8]:
sample_sizes_LB = 60
sample_sizes_UB = 300

In [9]:
results = {}
all_objectives = []
lower_bounds = []

for i in range(M_size):  
    v = []
    w1, w2 = generate_parameters(sample_sizes_LB)
    model = solve_optimization_LB((w1, w2), sample_sizes_LB)
    x1_value_lb = model.x1.value
    x2_value_lb = model.x2.value
    objective_value = value(model.obj)
    v.append(objective_value)
    all_objectives.extend(v)
    mean_objective = np.mean(v)
  
    results[i] = {
        'mean_objective': mean_objective,
        'x1': x1_value_lb,
        'x2': x2_value_lb
    }
    lower_bounds.append(mean_objective)
    
    print(f"\nReplication {i + 1} Results:")
    print("x1:", model.x1.value)
    print("x2:", model.x2.value)
    print("y1:")
    for s in range(sample_sizes_LB):
        print(f"  {model.y1[s].value}")
    print("y2:")
    for s in range(sample_sizes_LB):
        print(f"  {model.y2[s].value}")


print("\nObjective Function Values for Each Replication:")
for i, mean_objective in results.items():
    print(f"  Replication {i + 1}: {mean_objective}")


print("\nLowes value for All Replications:")
print(f"  Lowes value: {min(lower_bounds)}")

average_objective = np.mean(all_objectives)
print(f"\nAverage Objective Function Value: {average_objective}")

std_dev = np.std(all_objectives, ddof=1)
margin_of_error = z_score * std_dev / np.sqrt(M_size)  
lower_bound_ci = average_objective - margin_of_error
upper_bound_ci = average_objective + margin_of_error
print(f"Confidence Interval (CI) for Average Objective Function Value: ({lower_bound_ci}, {upper_bound_ci})")



Replication 1 Results:
x1: 0.784608775978708
x2: 3.51792143545332
y1:
  2.22943484566088
  0.551979201289561
  1.13404273287325
  0.368635033694952
  2.38388275358867
  0.0
  0.0
  1.61913143004816
  0.471814083246951
  1.85372509080359
  0.312216122879282
  0.943904451535787
  0.891065688115002
  2.04080562673944
  0.0
  1.93412662131353
  2.10601736736557
  2.66397001737513
  0.0041867439052541
  0.0804722811433696
  1.96844565439079
  1.16673919289267
  0.0
  0.0
  0.580161588291722
  0.0
  2.07454150543923
  0.0
  0.0
  2.18568973552488
  0.0
  0.0
  0.0
  1.60960365840811
  2.05356426778167
  1.43600386301496
  0.0
  2.37933240250976
  0.0
  2.58292291240642
  0.0
  1.93158311861336
  0.0
  0.0
  0.0
  2.98321250492814
  0.270737524856702
  0.87219877561436
  2.8932718774098
  2.16661819293306
  0.0
  0.470016617140171
  1.26108286532134
  0.0455126482895845
  1.66948961094547
  0.513978362430797
  0.525747231285345
  0.34072538498436
  1.88930864572151
  2.67831934719564
y2:
  0

In [10]:
results_ub = {}
all_objectives_ub = []  
detailed_ub_objectives = [] 
ub_cis = []  
upper_bounds = []  

for i in range(M_size):
    x1_value_lb = results[i]['x1']
    x2_value_lb = results[i]['x2']
    objectives_for_current_lb = []  

    for j in range(T_size):
        w1, w2 = generate_parameters(sample_sizes_UB)
        ub_model = solve_optimization_UB((w1, w2), sample_sizes_UB, x1_value_lb, x2_value_lb)
        objective_value_ub = value(ub_model.obj)
        objectives_for_current_lb.append(objective_value_ub)  
        detailed_ub_objectives.append((i, j, objective_value_ub))  

        print(f"\nLB Replication {i + 1}, UB Replication {j + 1} Results (Objective: {objective_value_ub}):")
        print(f"x1 (from LB): {x1_value_lb}, x2 (from LB): {x2_value_lb}")
        for s in range(sample_sizes_UB):
            print(f"  y1 UB {s + 1}: {ub_model.y1[s].value}, y2 UB {s + 1}: {ub_model.y2[s].value}")

    all_objectives_ub.extend(objectives_for_current_lb) 
    
    std_dev_ub = np.std(objectives_for_current_lb, ddof=1)
    margin_of_error_ub = z_score * std_dev_ub / np.sqrt(len(objectives_for_current_lb))
    lower_bound_ci_ub = np.mean(objectives_for_current_lb) - margin_of_error_ub
    upper_bound_ci_ub = np.mean(objectives_for_current_lb) + margin_of_error_ub
    ub_cis.append((i, lower_bound_ci_ub, upper_bound_ci_ub))

    mean_objective_ub = np.mean(objectives_for_current_lb)
    results_ub[i] = mean_objective_ub
    upper_bounds.append(mean_objective_ub)

# LB Optimization Results
print("\n--- LB Optimization Results ---")
print(f"Lowest Objective Value for All LB Replications: {min(lower_bounds)}")
average_objective_lb = np.mean(lower_bounds)
std_dev_lb = np.std(lower_bounds, ddof=1)
margin_of_error_lb = z_score * std_dev_lb / np.sqrt(M_size)
print(f"Average Objective Function Value (LB Problem): {average_objective_lb}")
print(f"Confidence Interval (CI) for Average Objective Function Value (LB Problem): ({average_objective_lb - margin_of_error_lb}, {average_objective_lb + margin_of_error_lb})")

print("\n--- UB Optimization Results ---")
average_ub_per_lb = {}  


for lb_rep, ub_rep, objective in detailed_ub_objectives:
    print(f"Objective Value for LB Replication {lb_rep + 1}, UB Replication {ub_rep + 1}: {objective}")
    
   
    if lb_rep not in average_ub_per_lb:
        average_ub_per_lb[lb_rep] = []
    average_ub_per_lb[lb_rep].append(objective)

print("\nAverage Objective Values for UB Replications in Each LB Batch:")
for lb_rep, objectives in average_ub_per_lb.items():
    average_obj = np.mean(objectives)
    print(f"Average Objective  of UB, in LB Replication {lb_rep + 1}: {average_obj}")

print("\nConfidence Intervals for UB Optimizations:")
for lb_rep, lower_ci, upper_ci in ub_cis:
    print(f"CI for average of UB, in LB Replication {lb_rep + 1}: ({lower_ci}, {upper_ci})")


LB Replication 1, UB Replication 1 Results (Objective: 6.310803060979923):
x1 (from LB): 0.784608775978708, x2 (from LB): 3.51792143545332
  y1 UB 1: 2.45977131170645, y2 UB 1: 0.588263823161834
  y1 UB 2: 2.75231917764833, y2 UB 2: 0.196169772951651
  y1 UB 3: 0.0, y2 UB 3: 0.0
  y1 UB 4: 0.580708672547199, y2 UB 4: 0.0
  y1 UB 5: 0.571195403975199, y2 UB 5: 0.0
  y1 UB 6: 1.68675803061094, y2 UB 6: 0.0
  y1 UB 7: 2.86460317619099, y2 UB 7: 0.353175852718912
  y1 UB 8: 0.39713296594166, y2 UB 8: 0.0
  y1 UB 9: 3.04983588347865, y2 UB 9: 0.0
  y1 UB 10: 2.28788170922382, y2 UB 10: 0.0
  y1 UB 11: 1.20940326843286, y2 UB 11: 0.0
  y1 UB 12: 2.85019906395554, y2 UB 12: 0.0
  y1 UB 13: 1.82270333009912, y2 UB 13: 0.0
  y1 UB 14: 0.0, y2 UB 14: 0.333604005695142
  y1 UB 15: 2.30668526152583, y2 UB 15: 0.449369103693361
  y1 UB 16: 0.0, y2 UB 16: 0.0
  y1 UB 17: 2.06404032966018, y2 UB 17: 0.0
  y1 UB 18: 1.4870223905112, y2 UB 18: 0.0
  y1 UB 19: 2.13846984408985, y2 UB 19: 0.0
  y1 UB 20

In [11]:
def ub_lb_gaps(average_ub_per_lb, lower_bounds):
    
    gaps = []

    print("\nGaps between Average UB Objectives and Average LB Objective:")

    for lb_rep, ub_objectives in average_ub_per_lb.items():
        average_ub = np.mean(ub_objectives)
        gap = average_ub - average_objective_lb
        gaps.append(gap)

        print(f"Gap between UB and LB in Replication {lb_rep + 1}: {gap}")

    return gaps

gaps = ub_lb_gaps(average_ub_per_lb, lower_bounds)


Gaps between Average UB Objectives and Average LB Objective:
Gap between UB and LB in Replication 1: 0.05422514907509868
Gap between UB and LB in Replication 2: 0.029568172204336385
Gap between UB and LB in Replication 3: 0.04331931774951059
Gap between UB and LB in Replication 4: 0.06928493507133826
Gap between UB and LB in Replication 5: 0.0375884480075559


**Question 4 answer**

When we use more data to do our calculations, the gap between our UB (Upper Bound) and LB (Lower Bound) predictions decrease. This means our UB and LB are getting tighter, giving us a clearer picture of what to expect. On the flip side, with less data, our UB and LB are farther apart, making our predictions less certain. 


By increasing the sample size, the length of the Confidence Intervals (CIs) tends to decrease. This shorter CI length indicates that our estimates become more precise, offering a narrower range of possible outcomes and thus increasing our confidence in the predictions made which make sense.